In [ ]:
import time

from enum import Enum
class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2


class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass


from abc import ABC


class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time


class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT


class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET


class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC
    

class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit = False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit
        



class MatchingEngine():
    def __init__(self):
        self.bid_book = []
        self.ask_book = []
        # These are the order books you are given and expected to use for matching the orders below

    # Note: As you implement the following functions keep in mind that these enums are available:
#     class OrderType(Enum):
#         LIMIT = 1
#         MARKET = 2
#         IOC = 3

#     class OrderSide(Enum):
#         BUY = 1
#         SELL = 2

    def handle_order(self, order):
        # Implement this function
        # In this function you need to call different functions from the matching engine
        # depending on the type of order you are given
        if order.type == OrderType.LIMIT:
            return self.handle_limit_order(order)
        elif order.type == OrderType.MARKET:
            return self.handle_market_order(order)
        elif order.type == OrderType.IOC:
            return self.handle_ioc_order(order)
        else:
            # You need to raise the following error if the type of order is ambiguous
            raise UndefinedOrderType("Undefined Order Type!")

    
    def handle_limit_order(self, order): 
        initial_quantity = order.quantity
        filled_orders = []
        
        if order.side == OrderSide.BUY:
            opposite_book = self.ask_book
        elif order.side == OrderSide.SELL:
            opposite_book = self.bid_book
        else:
            raise UndefinedOrderSide("Undefined Order Side!")


        while order.quantity > 0 and opposite_book:
            top_order = opposite_book[0]
            match = (order.side == OrderSide.BUY and order.price >= top_order.price) or \
                    (order.side == OrderSide.SELL and order.price <= top_order.price)
            if not match:
                break
            fill_quantity = min(order.quantity, top_order.quantity)

            if order.quantity <= top_order.quantity:
                filled_orders.append(FilledOrder(order.id, order.symbol, initial_quantity, order.price, order.side, order.time))
                filled_orders.append(FilledOrder(top_order.id, top_order.symbol, fill_quantity, top_order.price, top_order.side, top_order.time))

            elif order.quantity > top_order.quantity:
                filled_orders.append(FilledOrder(top_order.id, top_order.symbol, top_order.quantity, top_order.price, top_order.side, top_order.time))
                            
            order.quantity -= fill_quantity
            top_order.quantity -= fill_quantity

            if top_order.quantity == 0:
                opposite_book.pop(0)

        if order.quantity > 0:
            self.insert_limit_order(order) 
                 
        return filled_orders


    def handle_market_order(self, order):
        # Implement this function
        if order.side == OrderSide.BUY:
            opposite_book = self.ask_book
        elif order.side == OrderSide.SELL:
            opposite_book = self.bid_book
        else:
            raise UndefinedOrderSide("Undefined Order Side!")

        filled_orders = []

        i = 0
        while i < len(opposite_book) and order.quantity > 0:
            book_order = opposite_book[i]

            # Determine the quantity that can be matched
            matched_quantity = min(order.quantity, book_order.quantity)
            
            # Create a filled order
            filled_order = FilledOrder(book_order.id, book_order.symbol, matched_quantity, book_order.price, book_order.side, book_order.time)
            filled_orders.append(filled_order)

            # Update quantities of the matching order and the market order
            book_order.quantity -= matched_quantity
            order.quantity -= matched_quantity

            # Remove the book order if it's fully matched
            if book_order.quantity == 0:
                opposite_book.pop(i)
            else:
                i += 1

        # Market orders do not get inserted into the book, so no check for remaining quantity
        
        # The filled orders are expected to be the return variable (list)
        return filled_orders
        
        

    def handle_ioc_order(self, order):
        # Implement this function
        if order.side == OrderSide.BUY:
            opposite_book = self.ask_book
        elif order.side == OrderSide.SELL:
            opposite_book = self.bid_book
        else:
            raise UndefinedOrderSide("Undefined Order Side!")

        filled_orders = []

        i = 0
        while i < len(opposite_book) and order.quantity > 0:
            book_order = opposite_book[i]

            # Check if the order can be matched
            if (order.side == OrderSide.BUY and order.price >= book_order.price) or \
               (order.side == OrderSide.SELL and order.price <= book_order.price):
                matched_quantity = min(order.quantity, book_order.quantity)

                # Create a filled order
                filled_order = FilledOrder(book_order.id, book_order.symbol, matched_quantity, book_order.price, book_order.side, book_order.time)
                filled_orders.append(filled_order)

                # Update quantities
                book_order.quantity -= matched_quantity
                order.quantity -= matched_quantity

                # Remove fully matched orders from the book
                if book_order.quantity == 0:
                    opposite_book.pop(i)
                else:
                    i += 1
            else:
                i += 1

        # If the order was not fully matched, cancel the entire order
        if order.quantity > 0:
            return []

        return filled_orders


    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        # Implement this function
        # this function's sole puporse is to place limit orders in the book that are guaranteed
        # to not immediately fill
        # Determine the appropriate book based on the order side and insert the order
        if order.side == OrderSide.BUY:
            self.bid_book.append(order)
            self.bid_book.sort(key=lambda x: x.price, reverse=True)
        elif order.side == OrderSide.SELL:
            self.ask_book.append(order)
            self.ask_book.sort(key=lambda x: x.price)
        else:
            # Raise an error if the order side is ambiguous
            raise UndefinedOrderSide("Undefined Order Side!")

    def amend_quantity(self, id, quantity):
        # Implement this function
        # Hint: Remember that there are two order books, one on the bid side and one on the ask side
        for book in [self.bid_book, self.ask_book]:
            for order in book:
                if order.id == id:
                    # Ensure the new quantity is smaller than the existing quantity
                    if quantity >= order.quantity:
                        raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
                    order.quantity = quantity
                    return True

        return False
    
    def cancel_order(self, id):
        for book in [self.bid_book, self.ask_book]:
            for order in book:
                if order.id == id:
                    book.remove(order)
                    return True
        return False

import unittest

class TestOrderBook(unittest.TestCase):

    def test_insert_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        self.assertEqual(matching_engine.bid_book[0].quantity, 10)
        self.assertEqual(matching_engine.bid_book[0].price, 10)
    
    def test_handle_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        order_1 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(3, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        self.assertEqual(matching_engine.bid_book[0].price, 15)
        self.assertEqual(matching_engine.bid_book[1].quantity, 10)

        order_sell = LimitOrder(4, "S", 14, 8, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_limit_order(order_sell)

        self.assertEqual(matching_engine.bid_book[0].quantity, 6)
        self.assertEqual(filled_orders[0].id, 3)
        self.assertEqual(filled_orders[0].price, 15)
        self.assertEqual(filled_orders[1].id, 1)
        self.assertEqual(filled_orders[1].price, 10)
    
    def test_handle_market_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 6, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = MarketOrder(5, "S", 5, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_market_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(filled_orders[0].price, 10)

    def test_handle_ioc_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = IOCOrder(6, "S", 5, 12, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_ioc_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(len(filled_orders), 0)
    
    def test_amend_quantity(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.amend_quantity(2, 8)
        self.assertEqual(matching_engine.bid_book[0].quantity, 8)
    
    def test_cancel_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.cancel_order(1)
        self.assertEqual(matching_engine.bid_book[0].id, 2)

import io
import __main__
suite = unittest.TestLoader().loadTestsFromModule(__main__)
buf = io.StringIO()
unittest.TextTestRunner(stream=buf, verbosity=2).run(suite)
buf = buf.getvalue().split("\n")
for test in buf:
	if test.startswith("test"):
		print(test)

